# Multi-Target Regression

This notebook was made as part of a project for the course "Statistical Foundations of Machine Learning".
It has been included in this repository because it illustrates part of the experimentation phase of our research.
Of course, this notebook displays a very isolated and down-scaled version of the problem.
In reality, things were a lot more complex and not as perfectly structured as in this notebook.
Regardless, it gives some insight into how conclusions were drawn and substantiates the claim that we did, in fact, compare all approaches discussed in the methodology.
In particular, you will find a simple implementation of the stacked regressor, the results of which were not included in the thesis.

In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

# 1 - Problem Understanding

The research question covered in this notebook is inspired by Arno's master's thesis.
The aim of that thesis is to estimate ground reaction forces (**GRFs**), i.e. the forces exerted by the ground on a person's feet.
These estimations have to be based on measurements made by smart insoles (Moticon OpenGo) that can only measure pressure and inertia (at 2 ms intervals).

## 1.1 - The Dataset

To collect data, Arno had to wear the insoles and walk over force plates that accurately measure GRFs.
That's how `gait_cycles.csv` came about.

In [2]:
gait_cycles = pd.read_csv('gait_cycles.csv')

Each row in this dataset contains the 
- GRF of the left foot at timestamp $t$;
- GRF of the right foot at timestamp $t$.

These are the **target labels $Y$** of the ML problem.
Since we consider the world around us to be three-dimensional, these forces are measured along an $x$-, $y$- and $z$-axis.

In [3]:
LABELS = ['Fx_l', 'Fy_l', 'Fz_l', # GRF of left foot
          'Fx_r', 'Fy_r', 'Fz_r'] # GRF of right foot

Y = gait_cycles[LABELS]

Additionnally, each row contains the insole and sEMG signals
- at timestamp $t$
- at 6 previous timestamps (from $t - 2\mathrm{ms}$ until $t - 64\mathrm{ms}$)
- at 6 future timestamps (from $t + 2\mathrm{ms}$ until $t + 64\mathrm{ms}$)

These are the **input features $X$** of the ML problem.


In [4]:
def extract_features(df):
    X = df.drop(columns=LABELS, axis=1)      # drop labels
    return X.drop(columns=['trial'], axis=1) # drop trial 

X = extract_features(gait_cycles)

Finally, each row mentions the trial/choreagraphy in which this datapoint was measured.
Some trials were pretty simple (like "standing" or "forward walking"), while others were more complex (like "circular jogging").
We'll refer to this column as the **strata**.

In [5]:
strata = gait_cycles['trial']

## 1.2 - The Model

The aim is to train an artificial neural network (ANN) that outputs GRF estimations for both feet:
$ \vec{y} = 
    \begin{bmatrix}
        F{x_l} \\
        F{y_l} \\
        F{z_l} \\
        F{x_r} \\
        F{y_r} \\
        F{z_r}
    \end{bmatrix}
$.
This means we are dealing with a **multi-target regression model**, which gave rise to the research question discussed in this notebook.

With regard to the input of the model, we consider two approaches. In both cases, input vector $\vec{x}$ originates from the insole measurements of both feet (i.e. rows of $X$ in `gait_cycles.csv`).
1. In the first approach, these measurements will directly serve as input features of the model.
2. In the second approach the measurements will first be projected onto a lower dimensional space with PCA.

The following diagram gives a high-level overview of this process.

![A high-level diagram of the ML-problem at hand](images/problem_understanding.png)

#### Note
We've implemented the various versions of this model with the use of PyTorch.
This Python library makes use of a datastructure called **Tensor**.
For this reason we have to make conversions from NumPy arrays or Pandas DataFrames to Tensors, or vice-versa. 

In [6]:
X = torch.tensor(X.values, dtype=torch.float32)
Y = torch.tensor(Y.to_numpy().reshape((-1, 6)), dtype=torch.float32)

# 2 - Research Question

The common thread among the following experiments is the following question:

**What's the optimal approach for leveraging ANNs to tackle multi-target regression problems?**.

Although this fully captures the essence of this chapter, it should be noted that this formulation is not thorough enough to serve as a guiding research question.
It is imprecise for several reasons:
- First, it does not specify what can be considered as an "**approach**" or how it relates to the model. We have chosen to include PCA and multi-target methods.
- Second, it's unclear what is meant by "**optimal**". According to which metric will approaches be compared? We have chosen for out-of-sample error and will limit our scope to this particular dataset.
- Finally, what does it mean for a multi-target regression problem to be "**tackled**"? Special attention should be paid to the relationship between target variables.

For these reasons, the research question was split up into two sub-questions, each focussing on a seperate approach:

1. What's the impact of **PCA** on the network's in- and out-of-sample error?

2. How do different **multi-target regression methods** compare in terms of in- and out-of-sample error?

   We will consider:
    - single-target method
    - regressor Stacking
    - multi-task learning with hard parameter sharing

# 3 - Training/Validation/Test Split Strategy

We opted for a **stratified** 80/20 train/test-split.
With `gait_cycles['trials']` as the strata, the same proportion samples will be split for each gait_cycle.
We want the trained model to perform well on all types of gait trials.
This strategy ensures that the test set contains the necessary data to make that evaluation.
In an extreme counter scenario: a purely random split could result in a test set that doesn't include any data on "circular jogging".
Error measures computed on that test set would therefore not be representative for that gait trial.

The following image illustrates this strategy. 
For illustration purposes, the rightmost portion of each have trial has been shaded as test set.
Note, however, that these 80/20 splits **within** each stratum are **random**.

![Train-test-split](images/train-test-split.png)

In [7]:
from sklearn.model_selection import train_test_split

# Make the same stratified split for X, Y and strata
(X_full_train,      X_test, 
 Y_full_train,      Y_test, 
 strata_full_train, strata_test) = train_test_split(X, Y, strata, test_size=0.2, random_state=42, stratify=strata)

The same strategy will be followed for the train-validation split.
The full training set will be used for **cross-validation** over 5 stratified folds.

In [8]:
from sklearn.model_selection import StratifiedKFold

# Prepare 5 stratified folds for cross-validation
K = 5
kf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

def perform_pca(X_train, X_test):
    # Normalize features so their variances are comparable
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled  = scaler.transform(X_test)
    
    # Fit PCA model to the training data that captures 99% of the variance
    pca = PCA(n_components=0.99, svd_solver='full')
    pca.fit(X_train_scaled)

    # Project data from old features to principal components
    X_pc_train = pca.transform(X_train_scaled)
    X_pc_test  = pca.transform(X_test_scaled)
    
    # Convert the result to tensor
    X_train_tensor = torch.tensor(X_pc_train, dtype=torch.float32)
    X_test_tensor  = torch.tensor(X_pc_test,  dtype=torch.float32)

    return X_train_tensor, X_test_tensor

X_pc_full_train, X_pc_test = perform_pca(X_full_train, X_test)

Later on, we will have 2 versions for each of our multi-target regression methods.
In the first one, we'll use more of an end-to-end approach, and skip this PCA-function altogether.
In the second version, we will call this function on the relevant datasets before training the models.

# 4 - Principal Component Analysis

PCA is a form of dimensionality reduction used to simplify the input data of a model.
Its aim is to express the data with a new, smaller set of abstract variables, in a way that preserves as much of the original information as possible.
It does so by transforming each datapoint to a lower dimensional space.
The following illustration, found in "An Introduction to Statistical Learning" by James G., Witten D. , Hastie T., Tibshirani R. and Taylot J., serves as a handy visualization.

![](images/pca.png)

## 4.1 - Computation

*The following explanation was inspired by a book of Brunton & Kutz titled "Data Driven Science & Engineering - Machine Learning, Dynamical Systems, and Control".*

Let's consider an input set of $N$ training samples, each of which is a $d$-dimensional data point:
$$
X_{\mathrm{train}} =
    \begin{bmatrix}
        \vec{x}^\intercal_{0} \\
        \vec{x}^\intercal_{1} \\
        \vdots \\
        \vec{x}^\intercal_{N-1}
    \end{bmatrix}
\in \mathbb{R}^{N \times d}
$$

With this notation, we can think of each row as an observation and of each column as a variable.
We now want to construct a covariance matrix $C \in \mathbb{R}^{d \times d}$ that lists the covariance between any pair of these variables.
It is computed by:
1. Calculating the mean of all datapoints
    $\vec{\mu} = \frac{1}{N} \sum_{i = 0}^{N-1} {\vec{x}_i}$
2. Subtracting this mean from every sample in $X$:
    $$
    B :=
        \begin{bmatrix}
            \vec{x}^\intercal_{0} - \vec{\mu}^\intercal\\
            \vec{x}^\intercal_{1} - \vec{\mu}^\intercal\\
            \vdots \\
            \vec{x}^\intercal_{N-1} - \vec{\mu}^\intercal
        \end{bmatrix}
    \in \mathbb{R}^{N \times d}
    $$
3. Computing the matrix product $C := \frac{1}{n - 1} B^\intercal B$.

The principal components of $X_{\mathrm{train}}$ are now defined as the eigenvectors of $C$, for these reveal the directions of maximum variance in the data.
We can find them by computing the eigen-decomposition $C = P D P^\intercal$

with
$
P =
\begin{bmatrix}
    \vec{x}_{\lambda_0} & \vec{x}_{\lambda_1} & ... & \vec{x}_{\lambda_{d-1}}
\end{bmatrix}
\in \mathbb{R}^{d \times d}
$
and
$
D =
\begin{bmatrix}
    \lambda_0 & 0        & ... & 0     \\
    0         &\lambda_1 & ... & 0     \\
    \vdots    & \vdots   & \ddots & \vdots\\
    0         & 0        & ... & \lambda_{d-1}
\end{bmatrix}
\in \mathbb{R}^{d \times d}
$.

The eigenvalues $\lambda_{0}$, $\lambda_{1}$, ..., $\lambda_{d-1}$ in $D$ can be sorted largest to smallest.
By sorting their respective eigenvectors $\vec{x}_{\lambda_0}$, $\vec{x}_{\lambda_{1}}$, ..., $\vec{x}_{\lambda_{d-1}}$ in $P$ accordingly, we order them in terms of their contribution to the variance in the training set.
Let's say the first $n$ of them cover 99% of it.
By removing the last $d - n$ from $P$, we get a new matrix $P' \in \mathbb{R}^{d \times n}$ of $n$ principal components.

By multiplying the original training set $X_{\mathrm{train}}$ by $P'$ we transform it to a lower, $n$-dimensional space, while losing no more than 1% of variance.
This transformation is "stored" in $P'$ (i.e. it represents a matrix transformation), meaning it can be performed on other datasets, like the test set, as well.

## 4.2 - Implementation

To investigate the influence of PCA on the models' complexity and performance, we implemented the following function, using scikit-learn.
Before PCA, features are normalised so their variances can be compared in a "fair" way.
To avoid a form of data leakage, the PCA-model is fitted to the training set only.
Both training and test set are then transformed according to the same resulting transformation matrix.

# 5 - Multilayer Perceptron

As seen during the lectures, the multilayer perceptron is a type of feedforward neural network.
It's layers are fully connected, meaning each artifical neuron delivers its output to every neuron in the following layer.
In the following subsections, we will briefly restate the basic structure of the artificial neuron, as well as the MLP's learning algorithm: gradient descent.


## 5.1 - Artificial Neuron
Let there be $n \in \mathbb{N}_0$ input values:
$x_1, x_2, \dots, x_n \in \mathbb{R}.$

The artificial neuron computes a linear combination by multiplying each input $x_i$ with a weight $w_i \in \mathbb{R}$ and adding a bias term $b \in \mathbb{R}$.
We can write this bias as the product of an additional constant input $x_0 = 1$ with variable coefficient $w_0$, to allow for more elegant notation:
$$
\overbrace{w_0 x_0}^{b} + w_1 x_1 + w_2 x_2 + \dots + w_n x_n = \vec{w}^\intercal \vec{x} \in \mathbb{R}.
$$
with
$ \vec{w}^\intercal = \begin{bmatrix}
        w_{0} & w_{1} & \dots & w_{n}
    \end{bmatrix}
$
and
$ \vec{x} =
    \begin{bmatrix}
        x_{0} \\
        x_{1} \\
        \vdots \\
        x_{n}
    \end{bmatrix}
$.

This value is then passed through an nonlinear activation function $\sigma$ to produce a single output:
$\hat{y} = \sigma \left(  \vec{w}^\intercal \vec{x} \right)$.

In our implementation, we chose for a sigmoid activation, namely the logistic function:
$\sigma(x) = logistic(x) = \frac{1}{1 + e^{-x}}$


## 5.2 - Gradient Descent

Gradient descent starts by initializing the MLP's parameters with some random values $\vec{w}^{(0)} \in \mathbb{R}^{n+1}$.

The optimal values for $\vec{w}$ will be found by repeatedly executing the following update rule until convergence:

For $i = 0, 1, 2, ...$ compute:
$$\vec{w}^{(i+1)} := \vec{w}^{(i)} - \eta \nabla E(\vec{w}^{(i)}).$$

where $- \nabla E \in \mathbb{R}^{n+1}$ is the direction opposite to the gradient of the cost function and $\eta \in \mathbb{R}$ is the learning rate.

# 5 - Multilayer Perceptron

As seen during the lectures, the multilayer perceptron is a type of feedforward neural network.
It's layers are fully connected, meaning each artifical neuron delivers its output to every neuron in the following layer.
In the following subsections, we will briefly restate the basic structure of the artificial neuron, as well as the MLP's learning algorithm: gradient descent.


## 5.1 - Artificial Neuron
Let there be $n \in \mathbb{N}_0$ input values:
$x_1, x_2, \dots, x_n \in \mathbb{R}.$

The artificial neuron computes a linear combination by multiplying each input $x_i$ with a weight $w_i \in \mathbb{R}$ and adding a bias term $b \in \mathbb{R}$.
We can write this bias as the product of an additional constant input $x_0 = 1$ with variable coefficient $w_0$, to allow for more elegant notation:
$$
\overbrace{w_0 x_0}^{b} + w_1 x_1 + w_2 x_2 + \dots + w_n x_n = \vec{w}^\intercal \vec{x} \in \mathbb{R}.
$$
with
$ \vec{w}^\intercal = \begin{bmatrix}
        w_{0} & w_{1} & \dots & w_{n}
    \end{bmatrix}
$
and
$ \vec{x} =
    \begin{bmatrix}
        x_{0} \\
        x_{1} \\
        \vdots \\
        x_{n}
    \end{bmatrix}
$.

This value is then passed through an nonlinear activation function $\sigma$ to produce a single output:
$\hat{y} = \sigma \left(  \vec{w}^\intercal \vec{x} \right)$.

In our implementation, we chose for a sigmoid activation, namely the logistic function:
$\sigma(x) = logistic(x) = \frac{1}{1 + e^{-x}}$


## 5.2 - Gradient Descent

Gradient descent starts by initializing the MLP's parameters with some random values $\vec{w}^{(0)} \in \mathbb{R}^{n+1}$.

The optimal values for $\vec{w}$ will be found by repeatedly executing the following update rule until convergence:

For $i = 0, 1, 2, ...$ compute:
$$\vec{w}^{(i+1)} := \vec{w}^{(i)} - \eta \nabla E(\vec{w}^{(i)}).$$

where $- \nabla E \in \mathbb{R}^{n+1}$ is the direction opposite to the gradient of the cost function and $\eta \in \mathbb{R}$ is the learning rate.

## 5.3 - Implementation

We've made a generic implementation for the MLP using PyTorch.
It should be instantiated with a list of integers, denoting the number of neurons for any amount of hidden layers.
For example: `MLP([16, 32])`creates an MLP with 2 hidden layers. The first has 16 neurons, and the second has 32.
The in- and output layers are tailored to the number of features and target outputs of the training set.

In [10]:
from torch import nn
from torch.nn import Linear, Sigmoid, MSELoss

class MLP(nn.Module):
    """Generic implementation of a multiplayer perceptron (MLP) with sigmoid activation functions

    Args:
        hidden_sizes (list): list of #neurons in the hidden layer(s).
    """

    def __init__(self, hidden_sizes):
        super().__init__()
        self.hidden_sizes = hidden_sizes

        # Instantiate hidden layers with sigmoid activations
        hidden_layers = [Sigmoid()]
        for i in range(len(hidden_sizes) - 1):
            hidden_layers.append(Linear(hidden_sizes[i], hidden_sizes[i + 1]))
            hidden_layers.append(Sigmoid())

        self.__hidden_stack = nn.Sequential(*hidden_layers)

    def forward(self, X):
        '''Passes input features forward through the MLP's layers

        Args:
            X (torch.Tensor): input features

        Returns:
            torch.Tensor: output of the MLP
        '''
        X = self.__input_stack(X)
        X = self.__hidden_stack(X)
        X = self.__output_stack(X)
        return X

    def train_(self, X, Y):
        '''Trains the MLP on the training set [X, Y] through gradient descent

        Args:
            X (torch.Tensor): input features of the training set
            Y (torch.Tensor): target labels of the training set
        '''
        # Tailor in- and output layer to shape of X and Y
        self.__input_stack  = Linear(X.shape[1], self.hidden_sizes[0])
        self.__output_stack = Linear(self.hidden_sizes[-1], Y.shape[1])

        self.train() # train mode

        # Instantiate the optimizer
        optimizer = torch.optim.Adam(params=self.parameters(), lr=0.01)

        # Instantiate the loss function
        loss_function = MSELoss()

        # Training loop
        for epoch in range(100):
            optimizer.zero_grad() # don't accumulate gradients

            # Compute the loss and its gradient
            Y_pred = self(X)
            loss = loss_function(Y, Y_pred)
            loss.backward()  # back propagation

            optimizer.step() # adjust weights and biases accordingly


    def test(self, X, Y):
        '''Computes the MSE of predictions on the test set

        Args:
            X (torch.Tensor): input features of the test set
            Y (torch.Tensor): target labels of the test set

        Returns:
            float: MSE between predicted and target labels
        '''
        self.eval() # evaluation mode

        # Compare Y to predictions on X
        Y_pred = self(X)
        loss_function = MSELoss()
        mse = loss_function(Y, Y_pred).item()

        return mse

## 5.4 - Training the Model

In the following section we will train various models, each making use of the MLP in some way or another.
With the `train_` and `test` methods implemented above, we can easily train and score a model on a certain train-validation split.
This split is made 5-fold in the following `cross_validate` function:

In [11]:
def cross_validate(model, X, Y, strata, cv, pca=False):
    scores = []
    
    fold = 0
    for train_idx, val_idx in cv.split(X, strata):
        fold += 1

        # Make train-test split
        X_train, X_val = X[train_idx], X[val_idx]
        Y_train, Y_val = Y[train_idx], Y[val_idx]
        
        # Perform PCA if necessary
        if pca: X_train, X_val = perform_pca(X_train, X_val)

        # Train the model
        model.train_(X_train, Y_train)

        # Test the model
        mse = model.test(X_val, Y_val)
        scores.append(mse)

    return scores

We tuned the following **hyperparameters**, using Optuna:
- the number of hidden layers
- the number of neurons in each hidden layer

The following piece of code gives an example of how this was done:

In [12]:
import optuna

# 1. Define an objective function to be maximized
def objective(trial):

    # 2. Suggest values for the hyperparameters using a trial object
    ## a. Number of layers
    n_layers = trial.suggest_int('hidden_layers', 1, 2)
    
    ## b. Number of neurons per layer
    hidden_sizes = []
    for i in range(n_layers):
        size = trial.suggest_int(f'hidden_size_{i}', 2, 64)
        hidden_sizes.append(size)
    
    # 3. Instantiate a model with suggested hyperparameters
    model = MLP(hidden_sizes)
    
    # 4. Cross-validate the suggested model
    scores = cross_validate(model, X_full_train, Y_full_train[:, 0].reshape(-1, 1), strata_full_train, kf, pca=True)
    return np.mean(scores)

The creation of this study object was placed here in markdown, to avoid running it unnecessarily.

```python
# 5. Create a study object and optimize the objective function.
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
```

Each of the models in the following section are instantiated with a number of hidden neurons that came about through this process.

# 5 - Multi-Target Regression Methods

We will now consider three types of architectures for estimating the full GRFs.
Each architecture has a variant with and a variant without PCA.
Performance on the training and test set will be determined using this function:

In [13]:
def evaluate(model, X, Y):
    def error(Y, Y_pred):
        BODY_WEIGHT = 84 # kg
        loss_function = MSELoss()
        return np.sqrt(loss_function(Y, Y_pred).item()) * BODY_WEIGHT 

    # Make predictions
    Y_pred = model(X)
    
    # Build dictionary of errors
    errors = []
    
    # Overall error
    errors.append(error(Y, Y_pred))   
    
    # Error per target variable
    for i, label in enumerate(LABELS):
        y = Y[:, i].reshape(-1, 1)
        y_pred = Y_pred[:, i].reshape(-1, 1)
        errors.append(error(y, y_pred))
    
    return errors

We will plot them in a barchart with matplotlib.

In [14]:
def compare_errors(model1, model2):
    targets = ['Total', 'Fx_l', 'Fy_l', 'Fz_l', 'Fx_r', 'Fy_r', 'Fz_r']
    
    fig, axs = plt.subplots(1, 2, figsize=(12,3), sharey=True, sharex=True)
    x_axis = np.arange(len(targets)) 

    # Compute In-Sample Error
    errors1 = evaluate(model1, X_full_train, Y_full_train)
    errors2 = evaluate(model2, X_pc_full_train, Y_full_train)

    # Plot In-Sample Error
    axs[0].set_title("In-Sample Error") 
    axs[0].bar(x_axis - 0.2, errors1, 0.4, label = 'no PCA') 
    axs[0].bar(x_axis + 0.2, errors2, 0.4, label = 'PCA') 
    axs[0].set_xticks(x_axis, targets) 
    axs[0].set_xlabel("Target Variable") 
    axs[0].set_ylabel("MSE (in kg)") 
    axs[0].legend() 

    # Compute Out-Of-Sample Error
    errors1 = evaluate(model1, X_test, Y_test)
    errors2 = evaluate(model2, X_pc_test, Y_test)
    
    # Plot Out-Of-Sample Error
    axs[1].set_title("Out-Of-Sample Error") 
    axs[1].bar(x_axis - 0.2, errors1, 0.4, label = 'no PCA') 
    axs[1].bar(x_axis + 0.2, errors2, 0.4, label = 'PCA')
    axs[1].set_xlabel("Target Variable") 
    axs[1].yaxis.set_tick_params(labelleft=True)
    
    plt.show()

## 5.1 - Single-Target Method

The most straight-forward way of estimating multiple target variables, is by dedicating a seperate MLP to each one.
By doing so, we essentially treat the multi-target regression problem as six separate single-target regression problems.

The `SingleTargetRegressor` implemented in the following code block acts as no more than a container holding a submodel for each GRF component.
Each submodel is trained on the same input data, but on a different target label (see `train_`-method).
From then on, they can be used to make predictions for their dedicated target (see `forward`-method).
As such, the biggest shortcoming of this method immediately becomes apparent: it doesn't provide a means of exploiting the relationship between target variables.

In [15]:
class SingleTargetRegressor(nn.Module):
    """Implementation of the single-target method"""
    
    def __init__(self, sub_models):
        super().__init__()
        # Instantiate the submodels
        self.sub_models = sub_models

    def train_(self, X, Y):
        # Train a dedicated MLP for each target label
        self.sub_models['Fx_l'].train_(X, Y[:, 0].reshape(-1, 1))
        self.sub_models['Fy_l'].train_(X, Y[:, 1].reshape(-1, 1))
        self.sub_models['Fz_l'].train_(X, Y[:, 2].reshape(-1, 1))
        self.sub_models['Fx_r'].train_(X, Y[:, 3].reshape(-1, 1))
        self.sub_models['Fy_r'].train_(X, Y[:, 4].reshape(-1, 1))
        self.sub_models['Fz_r'].train_(X, Y[:, 5].reshape(-1, 1))
        
    def forward(self, X):
        # Make predictions for each component using the relevant MLP
        Fx_l = self.sub_models['Fx_l'](X)
        Fy_l = self.sub_models['Fy_l'](X)
        Fz_l = self.sub_models['Fz_l'](X)
        Fx_r = self.sub_models['Fx_r'](X)
        Fy_r = self.sub_models['Fy_r'](X)
        Fz_r = self.sub_models['Fz_r'](X)
        
        # Concatenate the predictions to a single output
        Y = torch.cat([Fx_l, Fy_l, Fz_l,
                       Fx_r, Fy_r, Fz_r], dim=1)
        return Y

The following submodels were the best ones considered during hyperparameter tuning.
For each model, Optuna suggested one or two hidden layers with a maximum of 64 neurons per layers.

In [16]:
##########################
# BEST MODEL WITHOUT PCA #
##########################
best_submodels = {
    'Fx_l': MLP([45]),     'Fy_l': MLP([58, 35]), 'Fz_l': MLP([59, 54]),
    'Fx_r': MLP([21, 24]), 'Fy_r': MLP([60, 32]), 'Fz_r': MLP([51, 46])
}
single_target_regressor = SingleTargetRegressor(best_submodels)
single_target_regressor.train_(X_full_train, Y_full_train)

In [17]:
##########################
# BEST MODEL WITH PCA #
##########################
best_submodels_pca = {
    'Fx_l': MLP([59, 24]), 'Fy_l': MLP([54, 32]), 'Fz_l': MLP([61]),
    'Fx_r': MLP([59, 23]), 'Fy_r': MLP([62, 57]), 'Fz_r': MLP([58])
}

single_target_regressor_pca = SingleTargetRegressor(best_submodels_pca)
single_target_regressor_pca.train_(X_pc_full_train, Y_full_train)

In [18]:
compare_errors(single_target_regressor, single_target_regressor_pca)

As we can see, the PCA-approach results in a lower MSE, both in-sample and out-of-sample.
The only exception is $F_y$ of the left foot.
The $z$-component of force gives the highest errors in both cases.
However, we should note that this is to be expected, given that this is the biggest component of the tree.
Relative to the magnitude of its force, the errors for $F_y$ are actually the highest.

## 5.2 - Regressor Stacking

Regressor stacking is an alternative approach, that will allow the predictions for components to learn from each other.
To do so, we'll start again by seperately training single-target **base regressors** for each component.
Their outputs ($\hat{y}_1^*$ to $\hat{y}_6^*$) serve as initial predictions that will be improved upon with the use of six **meta-models**. 
Each meta-model will be trained on the original features, as well as the predictions made by the base regressors.
As such, we'll dedicate one meta-model to each target variable.
The idea is that $\hat{y}_1^*$ to $\hat{y}_6^*$ can help the meta-models to correct the predictions of the base regressors.
In general, all estimations will be informed by the initial predictions for all target variables.
The following image illustrates this idea with a diagram of a simpler scenario with 2 target variables:

![test](images/regressor-stacking.png)


To see how this is useful, consider the following example.
Let $y_1$ and $y_2$ be the vertical GRF of the left and right foot, respectively.
Suppose that $\mathrm{Regressor}_1$ makes an estimation of $\hat{y}_1 = 0$, meaning that the left foot is off the ground.
If this prediction is correct, this implies that (in a normal walking trial) the right foot is bearing all the weight of the subject.
Through $\hat{y}_1$, $\mathrm{Regressor}_2$ can now take this into account when making a prediction for $y_2$.


In [19]:
class StackedRegressor(nn.Module):
    """Implementation of the multi-target regressor, using the single-target method"""
    
    def __init__(self, base_models, meta_models):
        super().__init__()
        
        # Instantiate MLP for each component
        self.base_regressor = SingleTargetRegressor(base_models)    
        self.meta_models = meta_models

    def forward(self, X):
        # Make intitial predictions with the base models
        Y_base = self.base_regressor(X)   
    
        # Extend X with these initial predictions
        X_meta = torch.cat([X, Y_base], dim=1)
    
        # Make final predictions for each component using the relevant meta model
        Fx_l = self.meta_models['Fx_l'](X_meta)
        Fy_l = self.meta_models['Fy_l'](X_meta)
        Fz_l = self.meta_models['Fz_l'](X_meta)
        Fx_r = self.meta_models['Fx_r'](X_meta)
        Fy_r = self.meta_models['Fy_r'](X_meta)
        Fz_r = self.meta_models['Fz_r'](X_meta)
        
        # Concatenate the final predictions to a single output
        Y_meta = torch.cat([Fx_l, Fy_l, Fz_l,
                            Fx_r, Fy_r, Fz_r], dim=1)
        return Y_meta

    def train_(self, X, Y):
        # Divide the training set into a half for the base models and a half for the meta models
        (X_base, X_meta,
         Y_base, Y_meta) = train_test_split(X, Y, test_size=0.5, random_state=42)
        
        # Train the base models
        self.base_regressor.train_(X_base, Y_base)

        # Add initial predictions to meta input set
        Y_base_pred = self.base_regressor(X_meta)
        X_meta_extended = torch.cat([X_meta, Y_base_pred], dim=1).detach()
        
        # Train each MLP on the relevant target label
        self.meta_models['Fx_l'].train_(X_meta_extended, Y_meta[:, 0].reshape(-1, 1))
        self.meta_models['Fy_l'].train_(X_meta_extended, Y_meta[:, 1].reshape(-1, 1))
        self.meta_models['Fz_l'].train_(X_meta_extended, Y_meta[:, 2].reshape(-1, 1))
        self.meta_models['Fx_r'].train_(X_meta_extended, Y_meta[:, 3].reshape(-1, 1))
        self.meta_models['Fy_r'].train_(X_meta_extended, Y_meta[:, 4].reshape(-1, 1))
        self.meta_models['Fz_r'].train_(X_meta_extended, Y_meta[:, 5].reshape(-1, 1))

To keep the overall complexity of the model similar to that of the `SingleTargetRegressor`, we limited all base and meta regressors to one hidden layer with a maximum of 64 neurons.
Before training, an arbitrary split was in the training data.
One half was used for training the base regressors, while the other half was used for training the meta models.

In [20]:
##########################
# BEST MODEL WITHOUT PCA #
##########################
best_base_models = {
    'Fx_l': MLP([50]), 'Fy_l': MLP([51]), 'Fz_l': MLP([61]),
    'Fx_r': MLP([40]), 'Fy_r': MLP([38]), 'Fz_r': MLP([53])
}

best_meta_models = {
    'Fx_l': MLP([33]), 'Fy_l': MLP([31]), 'Fz_l': MLP([51]),
    'Fx_r': MLP([58]), 'Fy_r': MLP([46]), 'Fz_r': MLP([59])
}

stacked_regressor = StackedRegressor(best_base_models, best_meta_models)
stacked_regressor.train_(X_full_train, Y_full_train)

In [21]:
#######################
# BEST MODEL WITH PCA #
#######################
best_base_models_pca = {
    'Fx_l': MLP([62]), 'Fy_l': MLP([54]), 'Fz_l': MLP([64]),
    'Fx_r': MLP([64]), 'Fy_r': MLP([61]), 'Fz_r': MLP([64])
}

best_meta_models_pca = {
    'Fx_l': MLP([43]), 'Fy_l': MLP([64]), 'Fz_l': MLP([59]),
    'Fx_r': MLP([50]), 'Fy_r': MLP([60]), 'Fz_r': MLP([63])
}

stacked_regressor_pca = StackedRegressor(best_base_models_pca, best_meta_models_pca)
stacked_regressor_pca.train_(X_pc_full_train, Y_full_train)

In [22]:
compare_errors(stacked_regressor, stacked_regressor_pca)

Again, PCA seems to have a positive influence on the errors.
Unfortunately, we notice that the goal of the stacking method has failed: errors didn't decrease compared to the previous method.

## 5.3 - Multi-Task Learning With Hard Parameter Sharing

Despite its complicated name, the final method is actually the simplest with regards to model architecture.
We can simply dedicate an output neuron to each of the target variables, leading to an output layer of 6 neurons.
By adapting the cost function to include predictions and targets of all 6 outputs, we can train them simultaneously.
This simultaneous learning is what's referred to as "multi-task learning".
It's not always clear when this is to the benefit and when this is to the detriment of the final predictions.
As a rule of thumb, it doesn't hurt if output variables are related to each other in some way, which we can assume is the case in our conext.
The "hard parameter sharing" refers to the fact that all output neurons are preceded by shared layers only.
Alternatively, one could construct an ANN where each output has its own "tail" of hidden layers that are not connected to the other outputs.
This would be referred to as "soft parameter sharing"

Originally, we tuned the hyperparameters of this model in the same way of the previous ones.
This gave us the following MLPs:

In [23]:
multi_task_regressor = MLP([64])
multi_task_regressor.train_(X_full_train, Y_full_train)

In [24]:
multi_task_regressor_pca = MLP([63, 47])
multi_task_regressor_pca.train_(X_pc_full_train, Y_full_train)

Once again, errors were higher than with the previous approach.
However, we figured that it might be unfair to limit this MLP to the same number of hidden neurons.
Previous methods resulted in a compound model.
Even though each of their submodels was limited to hidden layers of 64 neurons, together they offer a lot more compute.
We therefore rerun the hyperparameter tuning, but hidden neurons could now be suggested up until $6 \times 64$ neurons.
The results were the following:

In [25]:
multi_task_regressor = MLP([318, 77])
multi_task_regressor.train_(X_full_train, Y_full_train)

In [26]:
multi_task_regressor_pca = MLP([209])
multi_task_regressor_pca.train_(X_pc_full_train, Y_full_train)

In [27]:
compare_errors(multi_task_regressor, multi_task_regressor_pca)

There's a noticeable difference between the PCA and non-PCA variant, even more so than with the previous methods.
This time, the errors of the PCA model are comparable to those of the single-target method.
During our runs, they are even slightly lower.

# 6 - Conclusion

To conclude, we will make a final comparison between all of the previous approaches.

Let's start by addressing the first subquestion: **"What's the impact of PCA on the network's in- and out-of-sample error?"**

For each of the three types of architectures, the PCA-version resulted in lower errors, both in-sample and out-of-sample.
This might be because of several reasons.

- First, reducing the dimensionality of the input data makes the learning problem simpler in the sense that there are less input features.
  Since these preserve $99\%$ of the variance, we shouldn't lose any useful information in the process.
- Second, selecting the first $n$ principal components acts as a form of feature selection.
  The downside for us humans is that these new features are harder to interpret.
  An ANN on the other hand, works through abstract feature learning anyway.
  The linear transformation that is made by the PCA should not "bother" the MLP, because it works with linear combinations by design.
  In a sense, we give the network a "head start" by deliberately creating a highly informative linear combination of the variables in the original input data.
- Third, it's possible that PCA improved the results incidentally through the scaling or noise reduction.
  In that case it's not the inherent nature of principal components itself that reduced errors.
  This means other methods of preprocessing might give even better results.
  For example, the reason that PCA benefits this specific learning problem might be due to the multiple timestamps included in X.
  Other choices could have been made regarding this preprocessing.

Of course this doesn't mean that performing PCA is advisable in all deep learning contexts.
ANNs are known as an instrument for end-to-end learning.
The conclusions made in this notebook should not be overinterpreted onto other architectures and learning problems.
For instance, experimenting with other datasets, other hyperparameters or other implementations of gradient descent might lead to different conclusions.


Now, regarding the second subquestion: **"How do different multi-target regression methods compare in terms of in- and out-of-sample error?"**

We considered the single-target method, stacking and multi-task learning with hard parameter sharing.
For the complexities considered, the PCA version of the single-target method and the multi-task method gave comparable errors, both in-sample and out-of-sample.
The multi-task model even seems to get a slight edge in the runs we considered.

The stacking method gave the highest errors and did not succeed in its effort of exploiting the relationships between target variables.
This might be due to the fact that this relationship is very complex.
Indeed, datapoints have been collected in a wide variety of gait trials. 
Although one can imagine simple relationships between GRFs in an isolated trial (for example with the example of forward walking earlier on), it becomes way harder to do this when all trials are piled together.
To illustrate this, the following cell of code plots the correlation between target variables.
On the left, only the data of a single "forward walking" is included.
On the left we potted correlation for the entire dataset.

In [28]:
import seaborn as sns

fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Correlation plot for forward walking
corr1 = gait_cycles[gait_cycles['trial'] == 'FW_walking_01.csv'].iloc[:, :6].corr()
cmap = sns.diverging_palette(260, 20, s=75, l=40, n=5, center="light", as_cmap=True)
sns.heatmap(corr1, cmap=cmap, center=0, annot=True, fmt='.2f', square=True, robust=True, cbar=False, ax=axes[0])
axes[0].set_title('Forward Walking')

# Correlation plot for all trials
corr2 = gait_cycles.iloc[:, :6].corr()
sns.heatmap(corr2, cmap=cmap, center=0, annot=True, fmt='.2f', square=True, robust=True, cbar=False, ax=axes[1])
axes[1].set_title('All Trials')

# Show the plots
plt.show()

However, keep in mind that ANNs are capable of learning non-linear relationships as well.
For that reason, these correlation plots are by no means an indisputable justification for the lack of performance gains of stacking.

We conclude that the single-target method and multi-task method with hard parameter sharing perform similarly with regard to in- and out-of-sample error.
Because there's only a minor difference, it's hard to make conclusions on whether the latter succeeded at finding a relationship between the output variables.
The single target method is more approachable for ML engineers with limited to no experience in multi-output regression.
However, once you get familiar with the abstraction of training multiple output variables at once, the architecture of the multi-output method becomes easier to implement.
For data scientists who like an incremental approach, we'd suggest starting with a single target regressor for one of the target variables.
Once a decent model for that component has been found, one has a better intuition of the complexity that might be necessary for the multi-output regressor.